## System Setup

In [1]:
# Find CPU info
!grep -m1 'model name' /proc/cpuinfo | awk -F": " '{print $2}'
!grep 'cpu cores' /proc/cpuinfo | awk -F": " '{a[cores]+=$2}END{print "CPU cores: " a[cores]}'

Intel(R) Xeon(R) CPU @ 2.20GHz
CPU cores: 32


In [2]:
# Find Ram Info
!grep MemTotal /proc/meminfo | awk '{printf "%.1fGB RAM", $2 / 1024 / 1024}'

51.0GB RAM

In [3]:
!python3 --version

Python 3.10.12


In [4]:
# !pip install git+https://github.com/julien-hec/pyBKTR.git
!pip install pyBKTR

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.5 MB/s eta 0:00:00


## Library Setup

In [5]:
from pyBKTR.bktr import BKTRRegressor
from pyBKTR.examples.bixi import BixiData
from pyBKTR.kernels import KernelMatern, KernelParameter, KernelSE
from pyBKTR.tensor_ops import TSR
from pyBKTR.utils import simulate_spatiotemporal_data

# Following two librairies are for results manipulation
import numpy as np
import pandas as pd

## Mercator Projection (Appendix E)

In [6]:
import plotly.express as px

bixi_data = BixiData()

bktr_regressor = BKTRRegressor(
    data_df = bixi_data.data_df,
    spatial_positions_df = bixi_data.spatial_positions_df,
    temporal_positions_df = bixi_data.temporal_positions_df
)

print('# Initial dataframe (longitude, latitude) #')
print(bktr_regressor.geo_coords_projector.ini_df.head())
print()
print('# Mercator Projection dataframe ([-5, 5] scaled) #')
print(bktr_regressor.geo_coords_projector.scaled_ini_df.head())


# Initial dataframe (longitude, latitude) #
                                                     latitude  longitude
location                                                                
10002 - Métro Charlevoix (Centre / Charlevoix)      45.478228 -73.569651
4000 - Jeanne-d'Arc / Ontario                       45.549598 -73.541874
4001 - Graham / Brookfield                          45.520075 -73.629776
4002 - Graham / Wicksteed                           45.516937 -73.640483
5006 - Collège Édouard-Montpetit (de Gentilly /...  45.537226 -73.495067

# Mercator Projection dataframe ([-5, 5] scaled) #
                                                       lon_x     lat_y
location                                                              
10002 - Métro Charlevoix (Centre / Charlevoix)      1.035426 -1.583683
4000 - Jeanne-d'Arc / Ontario                       2.094213  2.298574
4001 - Graham / Brookfield                         -1.256364  0.692039
4002 - Graham / Wicksteed             

In [7]:
FIG_WIDTH = 550

fig_scale = px.scatter(
    bktr_regressor.geo_coords_projector.scaled_ini_df.reset_index(),
    x='lon_x', y='lat_y', hover_name='location',
    width = FIG_WIDTH
)
fig_scale.update_xaxes(range=[-5.5, 5.5])
fig_scale.update_yaxes(range=[-5.5, 5.5])
fig_scale.show()

fig_map = px.scatter_mapbox(
    bktr_regressor.geo_coords_projector.ini_df,
    lat='latitude', lon='longitude', zoom=9.9,
    mapbox_style='carto-positron',
    width = FIG_WIDTH
)
fig_map.show()
